In [60]:
import requests
import pprint

In [61]:
#1 The first step is to look for relevant table. In this case it is actually faster to paste the URL to chrome and do search function.
#URL is 'https://www150.statcan.gc.ca/t1/wds/rest/getAllCubesListLite'

#2. We found that the table we want is "Average satisfaction with life and with selected domains of life by age group and sex"
#3. What's important is the productId, which is be used for downloading the CSV.
#4. Product id is 13100106
#5. The next thing to do is to call the download method.
#https://www150.statcan.gc.ca/t1/wds/rest/getFullTableDownloadCSV/{product_id}/en

In [62]:
#Apply the GetFullTable Method
product_id = 13100106
download_path = f"https://www150.statcan.gc.ca/t1/wds/rest/getFullTableDownloadCSV/{product_id}/en"
response = requests.get(download_path)
table = response.json()
print(table)

{'status': 'SUCCESS', 'object': 'https://www150.statcan.gc.ca/n1/tbl/csv/13100106-eng.zip'}


In [63]:
#get download path
download_url = table["object"]
print(download_url)

https://www150.statcan.gc.ca/n1/tbl/csv/13100106-eng.zip


In [64]:
#Capture the file name with regex
import re

split= download_url.split("/csv/")
file_name = split[1]

file_name

'13100106-eng.zip'

In [65]:
# Create request to download...
r = requests.get(download_url, allow_redirects=True)

open(file_name, 'wb').write(r.content)

42335

In [66]:
#because the data is in zip format, use python to unzip it
from zipfile import ZipFile

with ZipFile(file_name, 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

In [67]:
# clean the data, get the columns we want
import pandas as pd
df = pd.read_csv("13100106.csv")

df.head()

,REF_DATE,GEO,DGUID,Age group,Sex,Satisfaction with life and with selected domains of life,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,2016,Canada,2016A000011124,"Total, 15 years and over",Both sexes,Life as a whole,Number,223,units,0,v119128399,1.1.1.1,7.9,NaN,NaN,NaN,1
1,2016,Canada,2016A000011124,"Total, 15 years and over",Both sexes,Standard of living,Number,223,units,0,v119128400,1.1.1.2,7.7,NaN,NaN,NaN,1
2,2016,Canada,2016A000011124,"Total, 15 years and over",Both sexes,Health,Number,223,units,0,v119128401,1.1.1.3,7.4,NaN,NaN,NaN,1
3,2016,Canada,2016A000011124,"Total, 15 years and over",Both sexes,Achievement in life,Number,223,units,0,v119128402,1.1.1.4,7.4,NaN,NaN,NaN,1
4,2016,Canada,2016A000011124,"Total, 15 years and over",Both sexes,Personal relationships,Number,223,units,0,v119128403,1.1.1.5,7.9,NaN,NaN,NaN,1


In [68]:
#extract the columns we want

df = df[["Age group","Sex","Satisfaction with life and with selected domains of life","VALUE"]]
df.drop_duplicates(keep = "first", inplace = True)
df.dropna(inplace= True)

In [69]:
#transform data extracted

In [70]:
df = df.rename(columns = {'Satisfaction with life and with selected domains of life':'LIFE_DOMAIN','Age group':'AGE_GROUP'})
df = df.rename(columns={'AGE_GROUP':'age_group', 'Sex':'sex', 'LIFE_DOMAIN':'life_domain', 'VALUE':'value'})

In [71]:

df.head()

,age_group,sex,life_domain,value
0,"Total, 15 years and over",Both sexes,Life as a whole,7.9
1,"Total, 15 years and over",Both sexes,Standard of living,7.7
2,"Total, 15 years and over",Both sexes,Health,7.4
3,"Total, 15 years and over",Both sexes,Achievement in life,7.4
4,"Total, 15 years and over",Both sexes,Personal relationships,7.9


In [52]:
df.to_csv('Output/statcan_csv.csv')

In [53]:
#create database connection and load database
from sqlalchemy import create_engine
connection_string = "postgres:1234@localhost:5432/happiness001_db"
engine = create_engine(f'postgresql://{connection_string}')

In [72]:
engine.table_names()

['gdp_change',
 'education',
 'healthy_pop',
 'comm_vitality',
 'democratic_engagement',
 'living_standards',
 'time_use',
 'environment',
 'leisure',
 'happiness_news',
 'life_domain']

In [73]:
df.to_sql(name='life_domain', con=engine, if_exists='append', index=True)